# Load dataset

In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset('wongnai_reviews')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/26.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/4.27M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6203 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review_body', 'star_rating'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['review_body', 'star_rating'],
        num_rows: 6203
    })
})

# Thai models
- airesearch/wangchanberta-base-att-spm-uncased
- clicknext/phayathaibert
- lst-nectec/HoogBERTa
- FacebookAI/xlm-roberta-base
- flax-community/gpt2-base-thai
- OpenThaiGPT
- Typhoon
- LLaMA 3.1
- Gemma 2
- Qwen 2

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "lst-nectec/HoogBERTa"

model = (
    AutoModelForSequenceClassification
    .from_pretrained(
        model_name,
        num_labels=5,
        problem_type="single_label_classification") # "multi_label_classification"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/575M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at lst-nectec/HoogBERTa and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
model = model.to("cuda")

In [ ]:
def tokenize(batch):
  return tokenizer(batch["review_body"], max_length=416, padding=True, truncation=True, return_tensors="pt")

In [ ]:
dataset_tokenized = dataset.map(tokenize, batched=True, batch_size=50)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6203 [00:00<?, ? examples/s]

In [ ]:
dataset_tokenized = dataset_tokenized.rename_column("star_rating", "label")
dataset_tokenized

DatasetDict({
    train: Dataset({
        features: ['review_body', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['review_body', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6203
    })
})

In [ ]:
# define performance metrics

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)

    return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import TrainingArguments, Trainer

batch_size = 16
logging_steps = len(dataset["train"]) // batch_size
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=10,
    save_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    disable_tqdm=False,
    logging_steps=logging_steps,
    log_level="error",
    report_to="none",
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["test"],
    args=training_args,
    compute_metrics=compute_metrics,
    )

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
10,No log,1.203646,0.479929,0.311274
20,No log,1.181442,0.479929,0.311274


KeyboardInterrupt: 

# PEFT

In [ ]:
!pip install trl bitsandbytes einops peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.0 MB/s eta 0:00:00


In [ ]:
!pip install trl bitsandbytes einops git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-7n12_jts
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-7n12_jts
  Resolved https://github.com/huggingface/peft.git to commit 8d9ecbed080eed79f32b279bdec211767919bd94
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.13.1.dev0-py3-none-any.whl size=329553 sha256=b52d439833715999c908b0c730a938174fb052ac4f4f66119cc9b9bc01cbe2b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-p_3nddjz/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.13.0
    Uninstalling peft-0.13.0:
      Successfully uninstalled peft-0.13.0


In [ ]:
from peft import LoraConfig
#, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForSequenceClassification

from trl import SFTTrainer

Overview of the supported task types:
- SEQ_CLS: Text classification.
- SEQ_2_SEQ_LM: Sequence-to-sequence language modeling.
- Causal LM: Causal language modeling.
- TOKEN_CLS: Token classification.
- QUESTION_ANS: Question answering.
- FEATURE_EXTRACTION: Feature extraction. Provides the hidden states which can be used as embeddings or features for downstream tasks.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "lst-nectec/HoogBERTa"

model = (
    AutoModelForSequenceClassification
    .from_pretrained(
        model_name,
        num_labels=5,
        problem_type="single_label_classification",
        device_map="auto"   # Distributes the model across available devices
    )
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(74905, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
# LoRA
peft_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=64,
      bias="none",
      task_type="SEQ_CLS",
      target_modules=[
          "query", "key", "value", "dense"
          ],
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=None,
    args=training_args,
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:195: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 512
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
10,No log,1.213046,0.479929,0.311274


KeyboardInterrupt: 

# Quantization

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "openthaigpt/openthaigpt-1.0.0-7b-chat"
#model_name = "lst-nectec/HoogBERTa"

model = (
    AutoModelForSequenceClassification
    .from_pretrained(
        model_name,
        num_labels=5,
        problem_type="single_label_classification",
        quantization_config=quantization_config,
        device_map="auto"   # Distributes the model across available devices
    )
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

NameError: name 'quantization_config' is not defined

# Llama Factory
https://colab.research.google.com/drive/1eRTPn37ltBbYsISy9Aw2NuI2Aq5CQrD9